In [6]:
import   pandas  as  pd
import   numpy   as  np
import   seaborn as  sb
import   matplotlib.pyplot as plt
from  sklearn.metrics         import   accuracy_score
from  sklearn.metrics         import   precision_score
from  sklearn.metrics         import   recall_score
from  sklearn.metrics         import   f1_score
from  sklearn.metrics         import   matthews_corrcoef
from  sklearn.metrics         import   classification_report  
from  sklearn.model_selection import   train_test_split
from  sklearn.model_selection import   GridSearchCV
from  sklearn                 import   metrics
from  statistics              import   mean
from  sklearn.preprocessing   import   StandardScaler  
from  sklearn.naive_bayes     import   GaussianNB
from  sklearn.neighbors       import   KNeighborsClassifier
from  sklearn.svm             import   SVC  
from  sklearn.neural_network  import   MLPClassifier
from  sklearn.tree            import   DecisionTreeClassifier
from  sklearn.ensemble        import   RandomForestClassifier
from  sklearn.ensemble        import   AdaBoostClassifier
from  sklearn.linear_model    import   LogisticRegression
####
class  TuneClass():                        
    def CreateDF_split(self, path):
        dataset = pd.read_csv(path)
        df  = pd.DataFrame(dataset) 
        dfd = df.drop(['uniprot'], axis=1)
        X   = dfd.drop(['target'], axis=1)
        y   = dfd['target'] 
        Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=1)
        scaler = StandardScaler()  
        scaler.fit(Xtrain)  
        Xtrain = scaler.transform(Xtrain)  
        Xtest  = scaler.transform(Xtest)  
        return  Xtrain, Xtest, ytrain, ytest
    ###
    def SetParamtere(self):
        nb = {
               'var_smoothing': np.logspace(0, -9, num=100)
        }
        knn = {
               'n_neighbors': np.arange(1, 16), 
               'metric'     : ['euclidean', 'manhattan', 'minkowski','mahalanobis'], 
               'weights'    : ['uniform', 'distance']  
        }
        svc = { 
               'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],          
               'C'     : [0.05, 0.1, 0.2, 0.3, 0.25, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
       }
        mlp = { 
               'hidden_layer_sizes': [(50, 50, 50), (50, 100, 50), (100,)],
               'solver'            : ['lbfgs', 'sgd', 'adam'],
               'activation'        : ['identity', 'logistic', 'tanh', 'relu'],
               'learning_rate'     : ['constant', 'adaptive', 'invscaling'],
               'alpha'             : [0.0001, 0.05]  
        }
        dt = { 
               'criterion'        : ['gini','entropy'],
               'splitter'         : ['best', 'random'],
               'max_depth'        : np.arange(2, 10),
               'min_samples_split': np.arange(0, 10),
               'min_samples_leaf' : np.arange(0, 10),
               'max_features'     : np.arange(0, 15)
        }
        rf = { 
               'criterion'        : ['gini', 'entropy'],
               'n_estimators'     : [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
               'max_depth'        : [3, 10, 20],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf' : [1, 2, 4],
               'max_features'     : ['auto', 'log2']
        }
        ada = { 
               'algorithm'        : ['SAMME', 'SAMME.R'],
               'n_estimators'     : [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]    
        }
        lr = {
               'solver' : ['newton-cg', 'lbfgs', 'liblinear'],
               'penalty': ['l1', 'l2', 'elasticnet', 'none'],
               'C'      : [0.01, 0.1, 1.0, 2.0]     
        }
        return  nb, knn, svc, mlp, dt, rf, ada, lr
    ###
    def Tune(self, model, params, Xtrain, ytrain):
        GS = GridSearchCV(model, param_grid=params, cv=10, n_jobs=-1)  
        GS.fit(Xtrain, ytrain)
        return GS
    
    def ComputeMesure(self, gs, Xtest, ytest):
        ypred = gs.predict(Xtest)
        a = round(accuracy_score(ytest, ypred), 2)
        p = round(mean(precision_score(ytest, ypred, average=None)), 2)
        r = round(mean(recall_score(ytest, ypred, average=None)), 2)
        f = round(mean(f1_score(ytest, ypred, average=None)), 2)
        m = round(matthews_corrcoef(ytest, ypred), 2)       
        return [a, p, r, f, m]
    ###
    def  ST(self, path):
        pnb, pknn, psvc, pmlp, pdt, prf, pada, plr = self.SetParamtere()
        Xtrain, Xtest, ytrain, ytest = self.CreateDF_split(path)
        print('Best parameters for NB:')
        gsNB = self.Tune(GaussianNB(), pnb, Xtrain, ytrain)
        bpNB = gsNB.best_params_
        print(bpNB)
        nb = self.ComputeMesure(gsNB, Xtest, ytest)
        print('Best parameters for KNN:')  
        gsKNN = self.Tune(KNeighborsClassifier(), pknn, Xtrain, ytrain)
        bpKNN = gsKNN.best_params_
        print(bpKNN)
        knn = self.ComputeMesure( gsKNN, Xtest, ytest)
        print('Best parameters for SVM:')
        gsSVC = self.Tune(SVC(), psvc, Xtrain, ytrain)
        print(gsSVC.best_params_)
        svc = self.ComputeMesure(gsSVC, Xtest, ytest)
        print('Best parameters for MLP:')
        gsMLP = self.Tune(MLPClassifier(), pmlp, Xtrain, ytrain)
        print(gsMLP.best_params_)
        mlp = self.ComputeMesure(gsMLP, Xtest, ytest)
        print('Best parameters for DT:')
        gsDT = self.Tune(DecisionTreeClassifier(), pdt, Xtrain, ytrain)
        print(gsDT.best_params_)
        dt = self.ComputeMesure(gsDT, Xtest, ytest)                
        print('Best parameters for RF:')
        gsRF = self.Tune(RandomForestClassifier(), prf, Xtrain, ytrain)
        print(gsRF.best_params_)
        rf = self.ComputeMesure(gsRF, Xtest, ytest)     
        print('Best parameters for ADA:')
        gsADA = self.Tune(AdaBoostClassifier(), pada, Xtrain, ytrain)
        print(gsADA.best_params_)
        ada = self.ComputeMesure(gsADA, Xtest, ytest)  
        print('Best parameters for LR:')
        gsLR = self.Tune(LogisticRegression(), plr, Xtrain, ytrain)
        print(gsLR.best_params_)
        lr = self.ComputeMesure(gsLR, Xtest, ytest)  
        return nb, knn, svc, mlp, dt, rf, ada, lr
    
    def Main(self, path):
        nb, knn, svc, mlp, dt, rf, ada, lr = self.ST(path)
        acc = [nb[0], knn[0], svc[0], mlp[0], dt[0], rf[0], ada[0], lr[0]] 
        pre = [nb[1], knn[1], svc[1], mlp[1], dt[1], rf[1], ada[1], lr[1]] 
        rec = [nb[2], knn[2], svc[2], mlp[2], dt[2], rf[2], ada[2], lr[2]] 
        f1  = [nb[3], knn[3], svc[3], mlp[3], dt[3], rf[3], ada[3], lr[3]] 
        mcc = [nb[4], knn[4], svc[4], mlp[4], dt[4], rf[4], ada[4], lr[4]] 
        all = [acc, pre, rec, f1, mcc]
        model = ['NB', 'KNN', 'SVM', 'MLP', 'DT', 'RF', 'ADA', 'LR']
        mes = ['Accuracy', 'Precision', 'Recall', 'F1', 'MCC']
        df = pd.DataFrame(all, index=mes, columns=model)
        return df
  
###        
obj = TuneClass()      
df = obj.Main('D:/GitHub-IBB/DATA/IF.csv')
df

Best parameters for NB:
{'var_smoothing': 0.3511191734215131}
Best parameters for KNN:
{'n_neighbors': 8}
Best parameters for SVM:
{'kernel': 'sigmoid'}
Best parameters for MLP:


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


{'hidden_layer_sizes': (100,)}
Best parameters for DT:
{'criterion': 'entropy'}
Best parameters for RF:
{'criterion': 'entropy'}
Best parameters for ADA:
{'algorithm': 'SAMME'}
Best parameters for LR:
{'solver': 'newton-cg'}


,NB,KNN,SVM,MLP,DT,RF,ADA,LR
Accuracy,0.76,0.80,0.77,0.80,0.76,0.83,0.85,0.87
Precision,0.75,0.80,0.78,0.79,0.75,0.83,0.84,0.87
Recall,0.74,0.79,0.74,0.79,0.75,0.82,0.83,0.87
F1,0.74,0.79,0.75,0.79,0.75,0.82,0.83,0.87
MCC,0.49,0.58,0.52,0.59,0.50,0.64,0.67,0.74
